In [1]:
import utils

In [5]:
from perceval.backends.core.meetup import Meetup

In [12]:
settings = utils.read_config_file('settings.yml')
es = utils.establish_connection(settings['es_host'])

Connection established


In [36]:
MAPPING_MEETUP = {
    "mappings": {
        "item": {
            "properties": {
                "date": {"type": "date"},
                "member_id": {"type": "keyword"},
                "member_name": {"type": "keyword"},
                "rsvp_response": {"type": "keyword"},
                "meetup_name": {"type": "keyword"},
                "meetup_id": {"type": "keyword"},
                "meetup_venue": {"type": "keyword"},
                "group": {"type": "keyword"}
            }
        }
    }
}

{'acknowledged': True, 'shards_acknowledged': True}

In [ ]:
es.indices.delete('meetup', ignore=[400, 404])
es.indices.create('meetup', body=MAPPING_MEETUP)

In [32]:
for group_name in settings['meetup']:
    group = Meetup(group=group_name, sleep_for_rate=True, api_token=None)
    items = []
    for item in group.fetch():
        items.append(item)

In [37]:
for item in items:
    for rsvp in item['data']['rsvps']:
        summary = {'date': rsvp['created'], 'member_id': rsvp['member']['id'], \
                   'member_name': rsvp['member']['name'], 'rvsp_response': rsvp['response'], \
                   'meetup_name': item['data']['name'], 'meetup_id': item['data']['id'], \
                   'meetup_venue":item['data']['venue']['name'], group': item['data']['group']['name']}
        es.index(index='meetups', doc_type='rvsp', body=summary)